# 데이터 연결
## 1. 라이브러리 설치
- 아나콘다에서 다음 라이브러리를 설치합니다. 다음 코드는 한줄씩 실행합니다.

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

###### - **pypdf**: 파이썬에서 PDF 파일을 다루기 위한 라이브러리입니다. PDF 파일을 읽거나 수정할 때 사용합니다.

In [ ]:
!pip install pypdf

###### - **tiktoken**: 오픈AI에서 제공하는 임베딩을 위한 라이브러리입니다. OpenAIEmbeddings을 사용하기 위해 필요합니다.

In [ ]:
!pip install tiktoken

###### - **faiss-cpu**: 페이스북(Facebook)의 AI 연구팀이 개발한 라이브러리로, 벡터의 유사도 검색을 위해 사용됩니다. 
###### - 만약 사용하는 컴퓨터가 GPU를 지원한다면 '!pip install faiss-gpu'로 설치해주세요

In [ ]:
!pip install faiss-cpu

###### - **sentence-transformers**: 자연어 처리에서 문장 또는 단락을 벡터로 변환하기 위해 사용되는 라이브러리입니다.

In [ ]:
!pip install sentence-transformers

###### - **langchain_community**: langchain_community는 langchain의 커뮤니티에서 개발한 모듈과 통합 기능들을 포함하는 라이브러리입니다. 
###### - 이 라이브러리를 통해 다양한 써드파티 서비스나 도구들과 쉽게 연동할 수 있습니다.
###### - 책에서 제공하는 원본 소스에  추가된 내용입니다.

In [ ]:
!pip install langchain_community

## 2. PDF 파일 불러와 보여주기
- 코드에서 document[5].page_content[:5000]의 의미는 PDF 6페이지에서 5,000글자를 읽어오라는 의미입니다. 
- 일반적으로 페이지에 대한 인덱스는 0부터 시작하므로 document[0]이 1페이지며, document[5]은 6페이지가 됩니다. 

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()
document[5].page_content[:5000]

## 3. 임베딩 처리
- 임베딩은 오픈 AI에서 제공하는 임베딩 모델을 사용하며, 벡터 데이터베이스로 파이스(FAISS)를 사용합니다.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-" #openai 키 입력

from langchain.vectorstores import FAISS
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings() #임베딩 처리
vectorstore = FAISS.from_documents(document, embeddings)
print(vectorstore)

- [테스트] 벡터 데이터베이스에 저장된 모든 청크들을 조회합니다. 

In [ ]:
# 모든 문서 조회
all_documents = vectorstore.similarity_search(query="", k=vectorstore.index.ntotal)

print(f"****** 크기: {vectorstore.index.ntotal} ")

# 출력할 때 인덱스를 포함하여 표시
for idx, doc in enumerate(all_documents):
    print(f"[{idx}][ {doc.page_content}]\n")

- [테스트] 특정 키워드로 조회한 결과 상위 2개를 보여줍니다.
###### - [langchain_core.vectorstores.VectorStore]( https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.similarity_search)
###### - [유사도 기반 검색 (Similarity search)](https://wikidocs.net/231578)

In [ ]:
query = 'Where did Tom go?'
docs = vectorstore.similarity_search(query,2)
for doc in docs:
    print(f"[ {doc.page_content}]\n")

## 4. 검색기(RetrievalQA) 활용 (rag-prompt)
###### - 원하는 질문에 답변할 수 있도록 RetrievalQA를 활용합니다. 
###### - 원문 pdf 파일이 영문임에도 불구하고 한글이 얼마나 잘 인식되는지 확인하고자 합니다.
###### - LLM 모델은 오픈AI에서 제공하는 gpt-4o-mini를 사용합니다.
> ###### "temperature=0": 이 값은 모델이 가장 가능성이 높은 응답을 선택하도록 하여, 매우 보수적이고 일관된 답변을 생성. 창의성은 거의 없고, 질문에 대해 매우 정해진 방식으로 답변
> ###### "temperature=1": 기본값으로, 적절한 수준의 창의성과 예측 가능성을 제공. 응답이 다채롭고 다양한 방식으로 나타날 수 있음


In [ ]:
# https://github.com/langchain-ai/langsmith-cookbook/blob/main/hub-examples/retrieval-qa-chain/retrieval-qa.ipynb
!pip install langchainhub

In [ ]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")
print(prompt)

In [ ]:
#from langchain.chat_models import ChatOpenAI # The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 # 창의성 (0.0 ~ 1.0) # 일반적으로 0.7 ~ 1.0 사용
                 model_name='gpt-4o-mini',  # 모델명 #gpt-4o-mini #gpt-3.5-turbo-16k
                )

from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever, 
    chain_type_kwargs={"prompt": prompt}
)
print(qa_chain)

In [ ]:
question = "Joe와 Tom은 누구인가요? 어디서 가져온 정보인가요? "
result = qa_chain.invoke({"query": question})
print(f"[ {result['result']}]\n")

In [ ]:
question = "그리고, 마을 무덤에 있던 남자를 죽인 사람은 누구인가요? 뒤 이야기도 들려주세요. 한글로 말해주세요. 어디서 가져온 정보인가요?"
result = qa_chain.invoke({"query": question})
print(f"[ {result['result']}]\n")

In [ ]:
question = "바이든이 누구인가요? 한글로 상세히 말해주세요. 고향과 나이와 가족관계는 어떻게 되나요? 어디서 가져온 정보인가요? LLM에서 가져온건가요?"
result = qa_chain.invoke({"query": question})
result["result"]

In [ ]:
question = "바이든이 누구인가요? 한글로 상세히 말해주세요. 바이든이 기르는 개의 이름은 무엇인가요? 어디서 가져온 정보인가요? LLM에서 가져온건가요?"
result = qa_chain.invoke({"query": question})
result["result"]

In [ ]:
question = "바이든이 누구인가요? 한글로 상세히 말해주세요. 고향과 나이와 가족관계는 어떻게 되나요? 바이든이 기르는 개의 이름은 무엇인가요? 어디서 가져온 정보인가요? LLM에서 가져온건가요?"
result = qa_chain.invoke({"query": question})
result["result"]

In [ ]:
question = "미국 대통령 바이든에 대해 설명해주세요. 어디서 가져온 정보인가요?"
result = qa_chain.invoke({"query": question})
print(f"[ {result['result']}]\n")

In [ ]:
question = "톰소여의모험이라는 소설에서 톰의 나이는 몇살인가요? 한글로 말해주세요. 어디서 가져온 정보인가요?"
result = qa_chain.invoke({"query": question})
print(f"[ {result['result']}]\n")

In [ ]:
question = "Hannibal, Missouri는 Young Samuel이 살았던 곳이지 Tom이 살던 곳은 아닙니다.Tom이 살고 있는 곳은 어디인가요? 한글로 말해주세요. 어디서 가져온 정보인가요?"
result = qa_chain.invoke({"query": question})
print(f"[ {result['result']}]\n")

## 5. 검색기(RetrievalQA) 활용 (chain_type="stuff")
###### - 원하는 질문에 답변할 수 있도록 RetrievalQA를 활용합니다. 
###### - 원문 pdf 파일이 영문임에도 불구하고 한글이 얼마나 잘 인식되는지 확인하고자 합니다.
###### - LLM 모델은 오픈AI에서 제공하는 gpt-4o-mini를 사용합니다.
> ###### "temperature=0": 이 값은 모델이 가장 가능성이 높은 응답을 선택하도록 하여, 매우 보수적이고 일관된 답변을 생성. 창의성은 거의 없고, 질문에 대해 매우 정해진 방식으로 답변
> ###### "temperature=1": 기본값으로, 적절한 수준의 창의성과 예측 가능성을 제공. 응답이 다채롭고 다양한 방식으로 나타날 수 있음


In [ ]:
#from langchain.chat_models import ChatOpenAI # The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정 # 창의성 (0.0 ~ 1.0) # 일반적으로 0.7 ~ 1.0 사용
                 model_name='gpt-4o-mini',  # 모델명 #gpt-4o-mini #gpt-3.5-turbo-16k
                )

from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever()

retrievalQA = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever)

print(retrievalQA)

In [ ]:
query = "Joe와 Tom은 누구인가요? 어디서 가져온 정보인가요? "
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")

In [ ]:
query = "그리고, 마을 무덤에 있던 남자를 죽인 사람은 누구인가요? 뒤 이야기도 들려주세요. 어디서 가져온 정보인가요?"
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")

In [ ]:
query = "톰소여의모험이라는 소설에서 톰의 나이는 몇살인가요? 어디서 가져온 정보인가요??"
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")

In [ ]:
query = "Hannibal, Missouri는 Young Samuel이 살았던 곳이지 Tom이 살던 곳은 아닙니다.Tom이 살고 있는 곳은 어디인가요? 어디서 가져온 정보인가요?"
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")

In [ ]:
query = "바이든이 누구인가요? 한글로 상세히 말해주세요"
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")

In [ ]:
query = "미국 대통령 바이든에 대해 설명해주세요"
result = retrievalQA.invoke({"query": query})
print(f"[ {result['result']}]\n")